In [1]:
import sys
stdout = sys.stdout
from hyperstream import HyperStream, UTC, TimeInterval
import numpy
import imp
from re import sub
import logging
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sys.stdout = stdout

import os
os.system('find -name *.pyc | xargs rm')

os.('..')

os.getcwd

%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set_style("dark")

minute = timedelta(minutes=1)
t1 = datetime(2016, 4, 28, 20, 0, 0, 0, UTC)
t2 = t1 + minute
ti = TimeInterval(t1, t2)

module_file_old = '/Users/td6301/dev/IRC-SPHERE_public/HyperStream/hyperstream/tools/histogram_from_list/2016-12-02_v0.0.2.py'
module_file_new = '/Users/td6301/dev/IRC-SPHERE_public/HyperStream/hyperstream/tools/histogram_from_list/2017-07-03_v0.0.3.py'

In [2]:
hs = HyperStream()

INFO:root:Reading /Users/td6301/dev/IRC-SPHERE_public/HyperStream/hyperstream_config.json
INFO:root:Updating channels
DEBUG:root:Processing devices


ChannelNotFoundError: Channel sphere_assets not found

In [ ]:
def load_module(module_file):
    module_file_components = module_file[:-3].split('/')

    with open(module_file, 'rb') as fp:
        module_name = '_'.join(map(lambda pp: sub(r'[^a-zA-Z0-9]', '_', pp), module_file_components))

        if module_name in sys.modules:
            print("module {} already loaded ...".format(module_name))
            del sys.modules[module_name]

        mod = imp.load_module(
            module_name, fp, module_file,
            ('.py', 'rb', imp.PY_SOURCE)
        )

        return mod

In [ ]:
# Generate some random date
dg = hs.plugins.data_generators

ticker = hs.channel_manager.memory.get_or_create_stream('ticker')
random = hs.channel_manager.memory.get_or_create_stream('random')
hs.tools.clock().execute(sources=[], sink=ticker, interval=ti)
dg.tools.random(seed=1234).execute(sources=[], sink=random, interval=ti, alignment_stream=ticker)

print(random.window().values()[:5])

In [ ]:
def plot_histograms(stream_old, stream_new, pts):
    data_old = stream_old.window().items()
    data_new = stream_new.window().items()
    
    # f, axarr = plt.subplots(len(data), sharex=True)
    for i, (old, new) in enumerate(zip(data_old, data_new)):
        print('{0:2.0f}, {1:2.0f}, {2}, {3}, {4}'.format(old.timestamp.minute, old.timestamp.second, old.value, new.value, np.array_equal(old.value, new.value)))
        # axarr[i].bar(pts, value)
        # axarr[i].set_title(timestamp)


In [ ]:
# Force loading of tool modules
mod_old = load_module(module_file_old)
mod_new = load_module(module_file_new)

breaks = list(np.linspace(-2, 2, 11))
# print(breaks, len(breaks))
pts = [-float('inf')] + list(map(np.mean, zip(breaks[:-1], breaks[1:]))) + [float('inf')]
# print(pts, len(pts))

h_old = mod_old.HistogramFromList(breaks=breaks)
h_new = mod_new.HistogramFromList(breaks=breaks)

hist_old = hs.channel_manager.memory.get_or_create_stream('hist_old')
hist_new = hs.channel_manager.memory.get_or_create_stream('hist_new')

hs.channel_manager.memory.purge_stream(hist_old.stream_id)
hs.channel_manager.memory.purge_stream(hist_new.stream_id)

h_old.execute(sources=[random], sink=hist_old, interval=ti)
h_new.execute(sources=[random], sink=hist_new, interval=ti)

plot_histograms(hist_old, hist_new, pts)

In [ ]:
# Force loading of tool modules
mod_old = load_module(module_file_old)
mod_new = load_module(module_file_new)

# breaks = list(np.linspace(-2, 2, 11))
# print(breaks, len(breaks))
# pts = [-float('inf')] + list(map(np.mean, zip(breaks[:-1], breaks[1:]))) + [float('inf')]
# print(pts, len(pts))

h_old = mod_old.HistogramFromList(break_width=0.1, first_break=-0.5, n_breaks=21)
h_new = mod_new.HistogramFromList(break_width=0.1, first_break=-0.5, n_breaks=21)

hist_old = hs.channel_manager.memory.get_or_create_stream('hist_old')
hist_new = hs.channel_manager.memory.get_or_create_stream('hist_new')

hs.channel_manager.memory.purge_stream(hist_old.stream_id)
hs.channel_manager.memory.purge_stream(hist_new.stream_id)

h_old.execute(sources=[random], sink=hist_old, interval=ti)
h_new.execute(sources=[random], sink=hist_new, interval=ti)

plot_histograms(hist_old, hist_new, pts)
